<a href="https://colab.research.google.com/github/ThanhNhann/Calculate-the-Screen-Time-of-Actors-in-the-movie/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import package 
import pandas as pd
import cv2
import math
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
#import keras.preprocessing import image
from keras.utils import np_utils
from skimage.transform import resize



Using TensorFlow backend.


In [2]:
#Read the video, extract frames from it and save them 
count = 0
cap = cv2.VideoCapture("Tom and jerry.mp4")
#get the fps 
frameRate = cap.get(5)

while(cap.isOpened()):
  #get the ID of current frame
  frameId = cap.get(1)
  ret, frame = cap.read()
  if (ret != True):
     break
  #One pic / 1s    
  if (frameId % math.floor(frameRate)==0):
      filename = "frame%d.jpg"%count
      count += 1
      cv2.imwrite(filename, frame)
cap.release()
print("Done")
      
  
      
  


Done


In [3]:
data = pd.read_csv('mapping.csv')     # reading the csv file
data.head()      # printing first five rows of the file

,Image_ID,Class
0,frame0.jpg,1
1,frame1.jpg,1
2,frame2.jpg,2
3,frame3.jpg,2
4,frame4.jpg,2


In [0]:
X = []
for img_name in data.Image_ID:
  img = plt.imread(img_name)
  X.append(img)
#converting list to array  
X = np.array(X)  

In [0]:
y = data.Class
#Onehot Encoding 
dummy_y = np_utils.to_categorical(y)

In [0]:
#resize image (we use VGG16 pretrained model with the shape(224,224,3))
image = []
for i in range(0, X.shape[0]):
  a = resize(X[i],preserve_range=True, output_shape=(224,224)).astype(int)
  image.append(a)
X = np.array(image)  

In [0]:
#preprocess before pass to model
from keras.applications.vgg16 import preprocess_input
X = preprocess_input(X, mode='tf')

In [0]:
#get train, valid set
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, dummy_y, test_size = 0.3, random_state = 42) 


In [9]:
#building the model
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout

base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224,224,3))

W0807 05:57:20.139609 140546040637312 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0807 05:57:20.175721 140546040637312 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0807 05:57:20.181930 140546040637312 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0807 05:57:20.220670 140546040637312 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



58892288/58889256 [==============================] - 1s 0us/step


W0807 05:57:21.854856 140546040637312 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0807 05:57:21.855904 140546040637312 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



In [10]:
X_train = base_model.predict(X_train)
X_valid = base_model.predict(X_valid)
X_train.shape, X_valid.shape

((208, 7, 7, 512), (90, 7, 7, 512))

In [0]:
X_train = X_train.reshape(208, 7*7*512)      # converting to 1-D
X_valid = X_valid.reshape(90, 7*7*512)

In [0]:
#preprocess the images and make them zero-centered which help the model to converge faster
train = X_train/X_train.max()   #centering the data
X_valid = X_valid/X_train.max()

In [0]:
#Build the model 
model = Sequential()
model.add(InputLayer((7*7*512,))) #input layer
model.add(Dense(units=1024, activation='sigmoid')) #hideen layer
model.add(Dense(3, activation='softmax')) #ouput layer

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 3075      
Total params: 25,694,211
Trainable params: 25,694,211
Non-trainable params: 0
_________________________________________________________________


In [0]:
#Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [24]:
#Training the model
model.fit(train, y_train, epochs=100, validation_data=(X_valid, y_valid))


Train on 208 samples, validate on 90 samples
Epoch 1/100
208/208 [==============================] - 2s 10ms/step - loss: 0.9879 - acc: 0.5529 - val_loss: 1.0264 - val_acc: 0.4111
Epoch 2/100
208/208 [==============================] - 0s 811us/step - loss: 0.4277 - acc: 0.8269 - val_loss: 0.9694 - val_acc: 0.4000
Epoch 3/100
208/208 [==============================] - 0s 775us/step - loss: 0.2151 - acc: 0.9423 - val_loss: 1.0113 - val_acc: 0.3667
Epoch 4/100
208/208 [==============================] - 0s 767us/step - loss: 0.1229 - acc: 0.9760 - val_loss: 0.9390 - val_acc: 0.4333
Epoch 5/100
208/208 [==============================] - 0s 781us/step - loss: 0.0639 - acc: 0.9904 - val_loss: 0.9293 - val_acc: 0.4000
Epoch 6/100
208/208 [==============================] - 0s 770us/step - loss: 0.0481 - acc: 0.9952 - val_loss: 0.9486 - val_acc: 0.3667
Epoch 7/100
208/208 [==============================] - 0s 773us/step - loss: 0.0496 - acc: 0.9808 - val_loss: 0.9205 - val_acc: 0.4222
Epoch 8/100

In [0]:
#Calculating the screen time 


In [40]:
count = 0
cap = cv2.VideoCapture("Tom and Jerry 3.mp4")
frameRate = cap.get(5)
while(cap.isOpened()):
  ret, frame = cap.read()
  frameId = cap.get(1)
  if (ret!=True):
    break
  if (frameId % math.floor(frameRate) == 0 ):
    namefile = "test%d.jpg" %count
    count += 1
    cv2.imwrite(namefile, frame)
cap.release()
print("Done")
print(count)
  


Done
185


In [0]:
test = pd.read_csv("test.csv")
test = test.drop(test.index[-1])

In [0]:
test_image = []
for img_name in test.Image_ID:
    img = plt.imread( img_name)
    test_image.append(img)
test_img = np.array(test_image)

In [0]:
test_image = []
for i in range(0,test_img.shape[0]):
    a = resize(test_img[i], preserve_range=True, output_shape=(224,224)).astype(int)
    test_image.append(a)
test_image = np.array(test_image)

In [53]:
test_image.shape

(185, 224, 224, 3)

In [0]:
# preprocessing the images
test_image = preprocess_input(test_image, mode='tf')

# extracting features from the images using pretrained model
test_image = base_model.predict(test_image)

# converting the images to 1-D form
test_image = test_image.reshape(185, 7*7*512)

# zero centered images
test_image = test_image/test_image.max()

In [0]:
#Make prediction
predictions = model.predict_classes(test_image)

In [56]:
print('Time of Jerry:', predictions[predictions==1].shape[0],"seconds")
print('Time of Tom:', predictions[predictions==2].shape[0],"seconds")

Time of Jerry: 8 seconds
Time of Tom: 112 seconds


(8,)